In [1]:
import numpy as np
import csv
import sys
import socket
from functools import partial
import time
import pandas as pd


path = r'C:\Users\barna\Documents\Feedback-Lockin-v2-master\scripts' #<------- your path here
sys.path.append(path)
import feedbackLockin_qcodes_module as fbl

from qcodes.instrument_drivers.tektronix.Keithley_2450 import Keithley2450

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\barna\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\barna\.qcodes\logs\220804-19484-qcodes.log


In [2]:
fbl = fbl.FeedbackLockin('fbl', TCPport=10000)

In [5]:
k2450 = Keithley2450('keithley', 'TCPIP0::192.168.236.102::5025::SOCKET')
k2450.source.function("voltage")
k2450.sense.function("current")
k2450.output_enabled(True)
k2450.sense.auto_range(True)
k2450.source.range(20)

Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04454022, firmware:1.6.7c) in 0.04s


The functions `fbl_first_outputCalibration`, `fbl_inputCalibration`, and, `fbl_outputCalibration` calibrate the feedback lock-in instrument. The first output channel is calibrated by measuring the current with a keithley source meter 2450 for a known applied output voltage. The gain of one input amplifier is measured, using the first calibrated output, by passing a known current through a known resistance and recording the pre-amplified potential. The 31 remaining output channels are calibrated from the first input, and the 31 remaining inputs are calibrated with a single output channel (other than the first). See [forthcoming] for a calibration setup picture/diagram.
<br>
<br>
The calibration data is automatically saved in csv files-- `C:\Users\barna\Documents\Feedback-Lockin-v2-master\scripts\fbl_channelGains.csv` and `C:\Users\barna\Documents\Feedback-Lockin-v2-master\scripts\fbl_currents.csv`-- so that the qcodes instrument driver can acess these values. 

In [16]:
def sample_until_precision_reached(fun_in,precision_in, N_init=50):
    '''
    samples data from a generic function, measure the standard deviation
    of samples and averages until a specified precision has been met. 
    Note, the precision is "one-sigma", so you may want a tighter bound.
    '''
    
    _std=np.infty
    N_tot=1
    first_step = True
    while _std/np.sqrt(N_tot)>precision_in:
        _new_samples = np.zeros(N_init)
        for i in range(N_init):
            _new_samples[i] = fun_in()        
        _N_new = len(_new_samples)
        
        if first_step == True:
            _samples=_new_samples
            _N_old = 0
            first_step = False
        else:
            _old_samples = _samples
            _N_old = len(_old_samples)
            _samples = np.zeros(_N_old+_N_new)
            _samples[0:_N_old]=_old_samples
            _samples[(_N_old-1):-1]=_new_samples
            
        _std = np.std(_samples)
        _mean = np.mean(_samples)
        N_tot = _N_old+_N_new
        
    return _mean, N_tot

def sample_amplifiedVin(ch):
    # gets amplified voltage from fbl's inputs and waits 1 second
    
    time.sleep(1)
    fbl.TCPdata.get()
    Vmeas= fbl.allData.get()[ch,2]
    
    return Vmeas
    
def measure_Iout(ch, Rbias_N):
    # measures the current through the specified output

    # get reference current from currents.csv
    path = 'C:/Users/barna/Documents/Feedback-Lockin-v2-master/scripts'
    Iout_ref = read_csv(path+'/fbl_currents.csv')
    Iref = Iout_ref[ch,Rbias_N]
                    
    # record current through output
    fbl.TCPdata.get()
    return fbl.getIout(ch, Iref)


def sample_Vin(ch, gain):
                    
    # record current through output
    time.sleep(1)
    fbl.TCPdata.get()
    return fbl.getVin(ch, gain)

def sample_keithleyI():
    
    time.sleep(1)
    return k2450.sense.current()

def read_csv(csvPath):
    # returns the specified csv as an array
    
    dataframe = pd.read_csv(csvPath, header=None)
    arr = dataframe.to_numpy()   
    return arr

    
def store_inCSV(csvPath, element, val):
    # stores val in the specified element of csv
    
    row = element[0]
    col = element[1]
    
    # gets array from csv and overwrites the specifed element
    arr = read_csv(csvPath)
    arr[row, col] = val
    
    # save updated array to same location
    pd.DataFrame(arr).to_csv(csvPath, index=False, header=False)
    
    
    
def fbl_inputCalibration(ch_N, gain_N, output_N=1, Rbias_N=3, RtoGround=923, precision=.001, Vout=7, save=True):
    '''
    AC calibration;
    calibrates ch with the specified gain setting; stores measured gain and phase offsets to fbl_channelGains.csv adn fbl_phases.csv
    source is the output channel connected to the input with the specified bias resistor setting
    R_toGround is the resistance from the current source to ground'''
    
    fbl.connectTCP()
    
    t0= time.time()
    
    # apply 7V to outputs and measure current
    fbl.setVout(output_N, Vout)
    time.sleep(5)
    Iout = measure_Iout(output_N, Rbias_N-1)
    
    sample_func =partial(sample_amplifiedVin, ch=ch_N)
    Vmeas, Nsamples  = sample_until_precision_reached(sample_func, precision) 
    
    # compute gain
    gain = Vmeas/(Iout *RtoGround)
    
    path = 'C:/Users/barna/Documents/Feedback-Lockin-v2-master/scripts'
    if save:
        store_inCSV(path+'/fbl_channelGains.csv', [ch_N, gain_N], gain)
    
    tf = time.time()
    fbl.closeTCP()
    
    print(f"Calibrated input ch{ch_N}, gain setting {gain_N}")
    print(f"calibrated gain: {gain}")
    print(f"Samples: {Nsamples}")
    print(f"Time: {tf-t0}")
    

def fbl_outputCalibration(ch_N, Rbias_N, input_N=30, gain_N=2, RtoGround=923, precision=.001, Vout=7, save=True):
    '''
    AC calibration;
    calibrates ch with the specified bias resistor setting; stores reference (at 1V) current to fbl.currents.csv'''
    
    fbl.connectTCP()
    
    t0= time.time()
    
    # apply 7V to outputs and measure current
    fbl.setVout(ch_N, Vout)
    time.sleep(5)
    
    # get ch gain from csv
    path = 'C:/Users/barna/Documents/Feedback-Lockin-v2-master/scripts'
    gains = read_csv(path+'/fbl_channelGains.csv')
    gain = gains[input_N, gain_N]
    
    sample_func =partial(sample_Vin, ch=input_N, gain=gain)
    Vin, Nsamples  = sample_until_precision_reached(sample_func, precision) 
    
    # compute reference current
    Iref = Vin/( RtoGround*Vout)
    
    if save:
        store_inCSV(path+'/fbl_currents.csv', [ch_N, Rbias_N-1], Iref)
    
    tf = time.time()
    fbl.closeTCP()
    
    print(f"Calibrated output ch{ch_N}, bias resistor setting {Rbias_N}")
    print(f"calibrated reference current: {Iref}")
    print(f"Samples: {Nsamples}")
    print(f"Time: {tf-t0}")
    
    
    
def fbl_first_calibration(Vout, precision=.001):
    '''
    DC calibration:
    calibrates ch with the specified bias resistor setting; when running this function
    on NImax, open NIPCIe-6738 "Dev1"; open test panels. Output 'Vout' Volts DC to the specified channel'''
      
    t0= time.time()
        
    # initialize keithley    
    k2450.output_enabled(True)
    k2450.source.function("voltage")
    k2450.source.voltage(1.)
    k2450.sense.function("current")
    sample_func = sample_keithleyI
    
    Imeas, Nsamples  = sample_until_precision_reached(sample_func, precision) 
    
    # the current produced by 1V
    
    tf = time.time()
    
    print(f'first output calibration; measured current:{Imeas}; sourced DC voltage: {Vout}')
    print(f"Samples: {Nsamples}")
    print(f"Time: {tf-t0}")
    
    return (-Imeas, Vout)
    
def fbl_save_first_calibration(ch_N, Rbias_N, Imeas_1, Imeas_2, Vout_1=5, Vout_2=-5):
    '''
    run this after taking two (Vout, Imeas) calibrations using the 'fbl_first_calibration' function;
    stores reference (at 1V) current, for the specified channel, resistor setting, to fbl.currents.csv;
    takes the difference of two output calibrations at different Vouts to mitigate offsets'''
    
    Iref_out = (Imeas_1-Imeas_2)/(Vout_1-Vout_2)
    
    path = 'C:/Users/barna/Documents/Feedback-Lockin-v2-master/scripts'
    store_inCSV(path+'/fbl_currents.csv', [ch_N, Rbias_N-1], Iref_out)
    
    print(f"Calibrated output ch{ch_N}, bias resistor setting {Rbias_N}")
    print(f"calibrated reference current: {Iref_out}")
    

In [11]:
#first output calibration

Imeas_1, Vout_1 = fbl_first_calibration(5)

first output calibration; measured current:-4.0088617e-08; sourced DC voltage: 5
Samples: 50
Time: 53.994975090026855


In [12]:
Imeas_2, Vout_2 = fbl_first_calibration(-5)

first output calibration; measured current:6.01372796e-08; sourced DC voltage: -5
Samples: 50
Time: 53.98250079154968


In [13]:
ch_N = 0
Rbias_N = 3

fbl_save_first_calibration(ch_N, Rbias_N, Imeas_1, Imeas_2, Vout_1=Vout_1, Vout_2=Vout_2)

Calibrated output ch0, bias resistor setting 3
calibrated reference current: 1.002258966e-08


In [30]:
#inputs

ch_N = 0
gain_N = 3

fbl_inputCalibration(ch_N,gain_N, output_N=0,Rbias_N=3, RtoGround=923.17, precision=.001, Vout=8, save=False)

connected to FBL TCP server
disconnected from FBL TCP server
Calibrated input ch0, gain setting 3
calibrated gain: 110106.07901195987
Samples: 50
Time: 57.83253502845764


In [19]:
# outputs

ch_N =0
Rbias_N = 3

fbl_outputCalibration(ch_N, Rbias_N, input_N=1, gain_N=2, RtoGround=923.17, precision=.001, Vout=10)

connected to FBL TCP server
disconnected from FBL TCP server
Calibrated output ch0, bias resistor setting 3
calibrated reference current: 0.0
Samples: 50
Time: 57.827256202697754


In [27]:
fbl.closeTCP()

disconnected from FBL TCP server
